### 1. 简单的示例 

<!-- ![image](docs/stream_warn.png) -->
<img src="docs/stream_warn.png" width="800" alt="stream warn">

In [ ]:
# 1.1 简单的示例

from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # pretrained YOLO11n model

# Run batched inference on a list of images
results = model([   "https://ultralytics.com/images/bus.jpg", 
                    "https://ultralytics.com/images/zidane.jpg"], stream=True)  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

### 2. 多种输入源

<!-- ![image](docs/source_type.png) -->
<img src="docs/source_type.png" width="800" alt="stream warn">


In [ ]:
# 2.1 截屏目标检测

from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Define current screenshot as source
source = "screen"

# Run inference on the source
results = model(source, stream=True)  # list of Results objects

results[0].show()

In [ ]:
# 2.2 摄像头目标检测

import cv2
from ultralytics import YOLO
import time

# Load the YOLO model
model = YOLO("yolo11n.pt")

# Open the default camera (0 is usually the built-in webcam)
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("错误: 无法打开摄像头")
    exit()

print("摄像头已成功打开。按 'q' 键退出。")
window_name = "YOLO 摄像头目标检测"

try:
    # Loop through the camera frames
    while True:
        # Read a frame from the camera
        success, frame = cap.read()
        
        if success:
            # Run YOLO inference on the frame
            results = model(frame)
            
            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            
            # Display the annotated frame
            cv2.imshow(window_name, annotated_frame)
            
            key = cv2.waitKey(30) & 0xFF
            if key == ord("q") or cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                break
        else:
            print("错误: 无法读取摄像头画面")
            break

finally:
    cv2.destroyAllWindows()
    cap.release()
    time.sleep(3)
    print("摄像头已释放，窗口已关闭。")

### 3. 推理参数

<!-- ![image](docs/model_predict.png) -->

<img src="docs/model_predict.png" width="800" alt="stream warn">

In [ ]:
# 3.1 推理参数分类

from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Run inference on 'bus.jpg' with arguments
results = model.predict(
                        # ---- 1. 输入配置与基础日志参数（定义数据源+控制运行日志）-------------------------------------------------------------------
                        source = "https://ultralytics.com/images/bus.jpg",  # 核心：指定推理数据源（图片路径/URL、视频、摄像头、目录等）【无固定默认值，需手动指定】
                        verbose = True,                                     # 默认值：True；辅助：控制终端是否打印详细推理日志（进度、结果统计等）
                        
                        # ---- 2. 数据预处理参数（对输入图像/视频进行预处理，适配模型输入）-------------------------------------------------------------------
                        imgsz = 320,                # 默认值：640；图像缩放：推理时的输入图像尺寸，平衡精度与速度
                        batch = 1,                  # 默认值：1；批量处理：指定推理批大小（仅对目录/视频/.txt文件有效），提升批量吞吐量
                        vid_stride = 1,	            # 默认值：1；视频跳帧：视频输入的帧步长，值越大跳帧越多，速度越快但时间分辨率降低
                        stream_buffer = False,      # 默认值：False；视频缓存：控制视频流帧的缓存策略（False丢弃旧帧适配实时，True排队帧避免遗漏）
                        rect = True,                # 默认值：True；矩形优化：（专属检测）矩形检测模式，减少图像填充冗余，优化预处理效率与检测精度
                        
                        # ---- 3. 模型推理与效率优化参数（控制模型推理的设备、速度、内存占用）-------------------------------------------------------------------
                        device = None,              # 默认值：None；设备指定：指定推理设备（cpu/cuda:0/0等），分配计算资源
                        half = False,               # 默认值：False；精度优化：启用FP16半精度推理，支持GPU的场景下提升速度，精度损失极小
                        compile = False,            # 默认值：False；编译优化：启用PyTorch 2.x torch.compile图形编译，优化推理速度（不支持则回退）
                        stream = False,             # 默认值：False；内存优化：启用流式处理，返回Results生成器，避免大批量/长视频加载占用过多内存
                        
                        # ---- 4. 检测结果筛选与后处理参数（对模型原始输出进行提纯，得到最终有效结果）-------------------------------------------------------------------
                        conf = 0.5,                 # 默认值：0.25；置信度过滤：设置最小置信度阈值，过滤低置信度无效检测结果
                        iou = 0.7,                  # 默认值：0.7；NMS阈值：设置非极大值抑制的IoU阈值，去除重叠度过高的重复检测框
                        max_det = 300,              # 默认值：300；最大检测数：限制单张图像/单帧视频的最大目标检测数量，避免密集场景输出过载
                        agnostic_nms = False,       # 默认值：False；类别无关NMS：合并不同类别间的重叠框，适配多类别目标重叠场景
                        classes = None,             # 默认值：None；类别筛选：按类别ID筛选结果，仅返回指定类别的检测目标
                        
                        # ---- 5. 结果输出与可视化参数（分为「持久化保存」和「实时窗口展示」，二者独立配置）-------------------------------------------------------------------
                        # 5.1 结果持久化保存（将结果存入本地文件，用于后续分析/归档）
                        save = True,                # 默认值：False（Python）/True（CLI）；核心保存开关：启用标注结果（图像/视频）的本地保存
                        project = None,             # 默认值：None；保存根目录：指定结果保存的项目根目录（仅save=True时有效）
                        name = None,                # 默认值：None；保存子目录：指定推理运行名称，在project下创建子目录区分结果（仅save=True时有效）
                        save_frames = False,        # 默认值：False；视频帧保存：处理视频时，将每帧标注结果单独保存为图像文件
                        save_txt = False,           # 默认值：False；文本结果保存：将检测结果以[class x_center y_center width height confidence]格式保存为.txt文件
                        save_conf = False,          # 默认值：False；置信度保存：配合save_txt=True，在txt文件中附带目标置信度分数
                        save_crop = False,          # 默认值：False；目标裁剪保存：将检测到的目标从原图中裁剪出来，单独保存为图像文件
                        
                        # 5.2 实时窗口可视化（仅推理时展示窗口，无持久化，用于调试/实时验证）
                        show = False,               # 默认值：False；窗口展示开关：启用弹出窗口，显示带标注的图像/视频帧
                        show_labels = True,         # 默认值：True；标签展示：在可视化窗口中显示目标类别标签
                        show_conf = True,           # 默认值：True；置信度展示：在类别标签旁显示目标检测置信度分数
                        show_boxes = True,          # 默认值：True；边界框展示：在可视化窗口中为目标绘制矩形边界框
                        line_width = None,          # 默认值：None；框线宽度：指定边界框线条宽度，None时自动适配图像尺寸
                        
                        # ---- 6. 高级功能扩展参数（非核心检测功能，用于调试/分割/下游任务）-------------------------------------------------------------------
                        visualize = False,          # 默认值：False；特征可视化：激活模型各层特征图的可视化与保存，用于调试和理解模型工作原理
                        augment = False,            # 默认值：False；测试时增强（TTA）：启用推理时的数据增强，提升检测鲁棒性，代价是降低推理速度
                        retina_masks = False,       # 默认值：False；高分辨率掩码：（专属分割）返回与原始图像尺寸匹配的高分辨率分割掩码
                        embed = None                # 默认值：None；特征提取：指定特征层，提取目标的特征向量/嵌入向量，用于聚类、相似性搜索等下游任务
                        )

for result in results:
    result.show()